# Bounds

## Input 
### A - numpy array, Adjacency Matrix
### X - numpy array, Node Features
### train_ids - numpy array, Train Node IDs. Make sure that Train ids range [0, V-1]
### nrounds_smooth - No of rounds of Smoothing. Use nrounds_smooth = 0 to use only raw node features. 
### L - No. of Layers of MLP. If Linear classifier use L = 1
### K - No of Classes. K = 2 for Binary classification
### W - list of parameters (numpy 2D arrays). Each element of the list is parameter of a layer of MLP. These parameters needn't be ordered. Just pass each layer parameter as an element of the list

In [1]:
import numpy as np

In [2]:
def compute_bounds(A, X, train_ids, nrounds_smooth, L, K, W):
    
    # Apply Feature Smoothing
    D = np.diag(np.sum(A, axis=1))
    D_1 = np.linalg.inv(D)
    Lp = np.matmul(D_1, (A + np.eye(A.shape[0])))
    for i in range(nrounds_smooth):
        X = np.matmul(Lp, X)
    
    # Compute distance to the Train nodes
    def get_agg_distance(Y):
        agg_distance = {}
        for i in range(A.shape[0]):
            agg_distance[i] = float('inf')
            for j in train_ids:
                agg_distance[i] = min(agg_distance[i],
                                      np.linalg.norm(Y[i] - Y[j]))
        return agg_distance
    
    distX = get_agg_distance(X)
    
    # Define constants in bounds
    ## Lipchitz parameter c - Not sure how to measure. Setting it to 1/max(em). Look at eq following eq.14 in the paper
    c = 1/max(distX.keys())
    ## N_o = No of Train nodes 
    N0 = train_ids.shape[0]
    ## alpha - There exists such a value! See Assumption 3 and discussion after Equation 15, 
    ## it governs probability of undesirable event. So should be set to high value i.e., 1/4
    alpha = 0.25 - 1e-6
    ## delta - Confidence, setting it 0.9
    delta = 0.9
    ## gamma - Margin setting it low value 1e-4
    gamma = 1e-4
    ## b - Apparently from a spectral norm inequality. I couldn't locate it in the reference. 
    ## They didn't even point the equation out!!
    b = 1
    ## C - Upper bound on Frenobius Norm of Wieghts "W_Fn". Setting it to ceil() of the max Frenobius norm, W_Fn
    W_Fn = np.sum([np.sum(np.square(w)) for w in W])**0.5
    C = np.ceil(W_Fn)
    ## Norm of the Smoothed Features
    B = np.linalg.norm(X, axis=1)
    B0 = np.max(B[train_ids])
    
    def compute_upper_bound(vi, dxi):
        ub = c * K * dxi
        ub += ((W_Fn*b) * (dxi**2) /(N0**alpha*(gamma/8)**2))
        ub += N0**(2*alpha-1)
        xb = (L*C*2*B[vi])/(delta*gamma)
        ub += N0**(-2*alpha) * np.log(xb)
        return ub
    
    gen_bounds = {}
    test_ids = np.setdiff1d(np.arange(A.shape[0]), train_ids)
    for vi in test_ids:
        gen_bounds[vi] = compute_upper_bound(vi, distX[vi])
    maxCS = max([gen_bounds[x] for x in gen_bounds])
    gen_bounds = {x: gen_bounds[x]/maxCS for x in gen_bounds}
    return gen_bounds
    

In [3]:
import torch as th
import networkx as nx
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import SGConv

[16:44:25] Using backend: pytorch
/opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/shreyshs/anaconda3/lib/python3.9/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory


# Example code

## Read the SGC model

In [4]:
import pandas as pd

In [5]:
import pickle

In [6]:
model = pd.read_pickle('CS-core-res/model.pkl')

In [7]:
W = [cp.detach().numpy() for cp in model.parameters()]
W = [np.column_stack(W)]

In [8]:
node_features = pickle.load(open('CS-core-res/node_features.pkl', 'rb'))

In [9]:
node_features.shape

(2708, 1433)

In [10]:
A = pickle.load(open('CS-core-res/adjacency_mat.pkl', 'rb'))

In [11]:
train_ids = pickle.load(open('CS-core-res/train_ids.pkl', 'rb'))

In [12]:
n_rounds_smooth = 2
n_layers = 1
n_labels = 7

In [13]:
gen_bounds = compute_bounds(A, node_features, train_ids, n_rounds_smooth, n_layers, n_labels, W)

In [14]:
len(gen_bounds)

2568

In [15]:
A.shape[0]

2708

In [16]:
len(train_ids)

140